In [29]:
!pip install TabPFN

Defaulting to user installation because normal site-packages is not writeable

   -------- ------------------------------- 1/5 [eval-type-backport]
   ---------------- ----------------------- 2/5 [einops]
   ---------------- ----------------------- 2/5 [einops]
   ---------------- ----------------------- 2/5 [einops]
   ---------------- ----------------------- 2/5 [einops]
   ---------------- ----------------------- 2/5 [einops]
   ------------------------ --------------- 3/5 [pydantic-settings]
   ------------------------ --------------- 3/5 [pydantic-settings]
   ------------------------ --------------- 3/5 [pydantic-settings]
   ------------------------ --------------- 3/5 [pydantic-settings]
   ------------------------ --------------- 3/5 [pydantic-settings]
   -------------------------------- ------- 4/5 [TabPFN]
   -------------------------------- ------- 4/5 [TabPFN]
   -------------------------------- ------- 4/5 [TabPFN]
   -------------------------------- ------- 4/5 [TabPFN]


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd,numpy as np,seaborn as sns,matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import fetch_california_housing
A,B = fetch_california_housing(return_X_y=True)

In [3]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

In [4]:
mpg = sns.load_dataset('mpg')

In [5]:
mpg = mpg.drop('name',axis=1)

In [6]:
mpg = pd.get_dummies(mpg,dtype='int64')
mpg.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin_europe,origin_japan,origin_usa
0,18.0,8,307.0,130.0,3504,12.0,70,0,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,0,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,0,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,0,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,0,0,1
5,15.0,8,429.0,198.0,4341,10.0,70,0,0,1
6,14.0,8,454.0,220.0,4354,9.0,70,0,0,1
7,14.0,8,440.0,215.0,4312,8.5,70,0,0,1
8,14.0,8,455.0,225.0,4425,10.0,70,0,0,1
9,15.0,8,390.0,190.0,3850,8.5,70,0,0,1


In [7]:
mpg['horsepower'].fillna(mpg['horsepower'].mean(),inplace=True)

C:\Users\bhuvp\AppData\Local\Temp\ipykernel_10020\3725346527.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mpg['horsepower'].fillna(mpg['horsepower'].mean(),inplace=True)


In [8]:
mpg.isnull().sum()

mpg              0
cylinders        0
displacement     0
horsepower       0
weight           0
acceleration     0
model_year       0
origin_europe    0
origin_japan     0
origin_usa       0
dtype: int64

In [9]:
X = mpg.drop('mpg',axis=1)
y = mpg['mpg']

In [10]:
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression,Lasso,Ridge,LinearRegression
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,StackingRegressor,VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from tabpfn import TabPFNRegressor
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
X.index

In [ ]:
level1_model = [
    ('lr',LinearRegression(n_jobs=-1)),
    ('svr',SVR()),
    ('ls',Lasso()),
    ('rg',Ridge()),
    ('rfr',RandomForestRegressor()),
    ('gbr',GradientBoostingRegressor()),
    ('knr',KNeighborsRegressor()),
    ('mlpr',MLPRegressor()),
    ('tbfnr',TabPFNRegressor()),
    ('lgbm',LGBMRegressor())
]
kf = KFold(n_splits=5,random_state=42,shuffle=True)
oof_preds = np.zeros((X_train.shape[0],len(level1_model)))
for i,(name,model) in enumerate(level1_model):
    fold_pred = np.zeros(X_train.shape[0])
    for (train_i,val_i) in kf.split(X_train):
        model.fit(X_train.iloc[train_i],y_train.iloc[train_i])
        fold_pred[val_i] = model.predict(X_train.iloc[val_i])
    oof_preds[:,i] = fold_pred

In [ ]:
oof_preds

In [ ]:
meta_X = oof_preds
meta_y = y_train

meta_model = [
    ('xgbr_meta',XGBRegressor(n_estimators=50,random_state=42)),
    ('mlp_meta',MLPRegressor(hidden_layer_sizes=(32,16),max_iter=300,random_state=42))
]
meta_pred = []
for name,model in meta_model:
    model.fit(meta_X,meta_y)
    meta_pred.append(model.predict(meta_X))
meta_pred = np.vstack(meta_pred).T

In [31]:
meta_y.shape

(318,)

In [26]:
final_pred = 0.6*meta_pred[:,0] + 0.4*meta_pred[:,1]

In [15]:
np.zeros((397,3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [37]:
print(f'Mean Squared Error = {mean_squared_error(meta_y,final_pred)}')
print(f'Mean Absolute Error = {mean_absolute_error(meta_y,final_pred)}')
print(f'Accuracy Score = {r2_score(meta_y,final_pred)}')

Mean Squared Error = 2.0446680442165084
Mean Absolute Error = 1.1286361362394148
Accuracy Score = 0.9673876995454467


In [34]:
xgb1 = XGBRegressor()
xgb1.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [36]:
r2_score(y_test,xgb1.predict(X_test))

0.8587019047531917